In [18]:
import numpy as np

def conv2d(image, kernel, stride=1):
    kernel_size = kernel.shape[0]
    output_size = (image.shape[0] - kernel_size) // stride + 1
    output = np.zeros((output_size, output_size))
    
    for y in range(0, output_size):
        for x in range(0, output_size):
            region = image[y:y+kernel_size, x:x+kernel_size]
            output[y, x] = np.sum(region * kernel)
    
    return output

In [19]:
def relu(feature_map):
    return np.maximum(0, feature_map)

In [20]:
def max_pooling(feature_map, size=2, stride=2):
    h, w = feature_map.shape
    output_h = (h - size) // stride + 1
    output_w = (w - size) // stride + 1
    output = np.zeros((output_h, output_w))
    
    for y in range(output_h):
        for x in range(output_w):
            region = feature_map[y*stride:y*stride+size, x*stride:x*stride+size]
            output[y, x] = np.max(region)
    
    return output

In [21]:
def softmax(x):
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum()

In [22]:
def cross_entropy(pred, label):
    return -np.log(pred[label] + 1e-9)

In [23]:
class SimpleCNN:
    def __init__(self):
        # Init random kernels (3x3)
        self.kernel = np.random.randn(3, 3) * 0.1
        self.fc_weights = np.random.randn(225, 3) * 0.1  # 15x15 output dari pooling
        
    def forward(self, image):
        self.conv_out = conv2d(image, self.kernel)
        self.relu_out = relu(self.conv_out)
        self.pool_out = max_pooling(self.relu_out)
        self.flat = self.pool_out.flatten()
        self.fc_out = np.dot(self.flat, self.fc_weights)
        self.out = softmax(self.fc_out)
        return self.out

    def backward(self, image, label, lr=0.01):
        # Forward pass
        output = self.forward(image)
        loss = cross_entropy(output, label)

        # Grad for fully connected
        d_out = output
        d_out[label] -= 1  # dL/dy_pred

        d_fc_weights = np.outer(self.flat, d_out)
        self.fc_weights -= lr * d_fc_weights

        # Grad untuk kernel dll bisa ditambahkan (cukup rumit tapi bisa)
        return loss

In [24]:
import os
import cv2

def load_images(folder, label, size=(32, 32)):
    data = []
    for fname in os.listdir(folder):
        path = os.path.join(folder, fname)
        img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
        if img is None:
            print(f"⚠️ Gagal baca gambar: {path}")
            continue  # skip file ini kalau nggak bisa dibaca
        img = cv2.resize(img, size)
        img = img / 255.0  # normalize
        data.append((img, label))
    return data

In [25]:
model = SimpleCNN()

# Load dataset
data = load_images("data/senang", 0) + load_images("data/sedih", 1) + load_images("data/marah", 2)
np.random.shuffle(data)

⚠️ Gagal baca gambar: data/sedih\crying-sad.gif


In [26]:
# Train
for epoch in range(10):
    total_loss = 0
    for img, label in data:
        loss = model.backward(img, label)
        total_loss += loss
    print(f"Epoch {epoch+1}, Loss: {total_loss:.4f}")

Epoch 1, Loss: 290.6377
Epoch 2, Loss: 289.9374
Epoch 3, Loss: 289.2584
Epoch 4, Loss: 288.5995
Epoch 5, Loss: 287.9595
Epoch 6, Loss: 287.3372
Epoch 7, Loss: 286.7316
Epoch 8, Loss: 286.1418
Epoch 9, Loss: 285.5669
Epoch 10, Loss: 285.0060
